## 1. 라이브러리 로드

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

## 2. 대표폰트/클러스터별 폰트 로드

In [19]:
fonts = pd.read_csv('대표폰트 및 클러스터별 폰트(유클리드거리).csv')
fonts

,0,1,2,3
0,GOYANGDEOGYANG B,KoPubWorld Dotum Medium,KoPubWorld Batang Light,netmarbleM
1,10X10,12롯데마트드림Light,Arita-buriB,20950870_경기천년바탕TTF_Bold
2,10X10Bold,12롯데마트드림Medium,Arita-buriL,20950871_경기천년바탕TTF_Regular
3,12롯데마트드림Bold,20950914_빛고을광주_Light_TTF,Arita-buriM,20950938_Batang_TTF
4,12롯데마트행복Bold,20956072_SuncheonR_TTF,Arita-buriSB,20956073_SuncheonB_TTF
...,...,...,...,...
73,이순신돋움체B,NaN,NaN,NaN
74,이순신돋움체M,NaN,NaN,NaN
75,한수원_한돋음_B,NaN,NaN,NaN
76,한수원_한돋음_R,NaN,NaN,NaN


In [20]:
repres_fonts = fonts.loc[[0]]
fonts_foreach_cluster = fonts.iloc[1:, :]
fonts_foreach_cluster

,0,1,2,3
1,10X10,12롯데마트드림Light,Arita-buriB,20950870_경기천년바탕TTF_Bold
2,10X10Bold,12롯데마트드림Medium,Arita-buriL,20950871_경기천년바탕TTF_Regular
3,12롯데마트드림Bold,20950914_빛고을광주_Light_TTF,Arita-buriM,20950938_Batang_TTF
4,12롯데마트행복Bold,20956072_SuncheonR_TTF,Arita-buriSB,20956073_SuncheonB_TTF
5,12롯데마트행복Light,Atomy-Light,BareunBatangB,20958935_UTOIMAGE_유토이미지고딕_R_TTF
...,...,...,...,...
73,이순신돋움체B,NaN,NaN,NaN
74,이순신돋움체M,NaN,NaN,NaN
75,한수원_한돋음_B,NaN,NaN,NaN
76,한수원_한돋음_R,NaN,NaN,NaN


## 3. 텍스트 렌더링 함수

In [38]:
from PIL import Image, ImageDraw, ImageFont
import os

def make_image(message, font_list):

    # font setting
    font_color = 'rgb(0, 0, 0)'
    font_size = 60
    
    # image size
    width = len(message) * font_size + 10
    height = font_size + 35
    bg_color = 'rgb(255, 255, 255)'
    x_margin = 5
    y_margin = 5
    
    # 렌더링 결과 파일 저장할 폴더 생성
    img_save_path = 'C:/Users/SM/Desktop/2022-2 졸업프로젝트/Similar font recommendation/' + message + '/'
    os.mkdir(img_save_path)
    
    for font in font_list:
      image =Image.new('RGB', (width, height), color = bg_color)
      font_name = font.replace('.ttf', '').replace('.TTF', '')
      font = ImageFont.truetype('C:/Users/SM/dir_/FontProject/_석사코드정리_학제간연구용/1. 폰트파일-_이미지파일 변환 코드/ttf_to_jpg/ttf_standard_font(183)/' + font, font_size)
      draw = ImageDraw.Draw(image)      
      draw.text((x_margin, y_margin), message, font=font, fill=font_color)
      
      # save file
      image.save(img_save_path + font_name + '.png')

## 4. 입력이미지 특징벡터 추출 

In [81]:
test_img_input_path = 'C:/Users/SM/Desktop/2022-2 졸업프로젝트/Similar font recommendation/훈민정음.png'

test_filename = os.fsdecode(test_img_input_path)
test_img = Image.open(os.path.join(input_path, test_filename)).convert('RGB')
test_img_vector = img2vec.get_vec(test_img)
    
test_img_vector_df = pd.DataFrame(test_img_vector).transpose()
test_img_vector_df

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,1.134893,0.185287,0.011392,0.0,0.010423,0.032565,0.096117,0.665791,0.0,0.005898,...,1.090696,1.475597,0.002462,0.044556,1.387439,0.0,0.504765,0.239854,0.075132,0.723508


## 5. 1차 유사도 비교

### 5.1 대표폰트로 텍스트 렌더링

In [39]:
font_list = os.listdir('C:/Users/SM/dir_/FontProject/_석사코드정리_학제간연구용/1. 폰트파일-_이미지파일 변환 코드/ttf_to_jpg/ttf_standard_font(183)')

repres_font_list = []
for font_idx in repres_fonts:
  repres_font_list.append(repres_fonts[font_idx].values[0] + '.ttf')

make_image("훈민정음", repres_font_list)


### 5.2 렌더링한 텍스트 이미지의 특징벡터 추출

In [111]:
import sys
sys.path.append("C:/Users/SM/dir_/FontProject/_석사코드정리_학제간연구용/3. 유사폰트추천코드/img2vec/img2vec_pytorch")  # Adds higher directory to python modules path.
from img_to_vec import Img2Vec

img2vec =Img2Vec(model="inception")
input_path = 'C:/Users/SM/Desktop/2022-2 졸업프로젝트/Similar font recommendation/훈민정음/'
list_pics1 = []
filenames1 = []
for file1 in os.listdir(input_path):
    filename1 = os.fsdecode(file1)
    img1 = Image.open(os.path.join(input_path, filename1)).convert('RGB')
    list_pics1.append(img1)
    filenames1.append(filename1)
    
vectors1 = img2vec.get_vec(list_pics1)
pics1 = {}
for i1, vec1 in enumerate(vectors1):
    pics1[filenames1[i1]] = vec1
    
rendering_1st_vector_df = pd.DataFrame(pics1).transpose()
rendering_1st_vector_df

AdaptiveAvgPool2d(output_size=(1, 1))
(4, 2048)


,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
GOYANGDEOGYANG B.png,0.055564,0.115681,0.000000,0.071439,0.128027,0.000000,0.025343,0.784571,0.000000,0.298496,...,0.412049,0.474661,0.231373,0.308604,1.170114,0.018534,0.167823,0.217642,0.000000,0.426701
KoPubWorld Batang Light.png,0.572750,0.095026,0.028765,0.149890,0.047640,0.017186,0.118919,1.753030,0.107149,0.766331,...,0.885373,0.108349,0.004951,0.446789,0.778579,0.010587,0.078890,0.096158,0.000829,0.996581
KoPubWorld Dotum Medium.png,0.345921,0.172748,0.000000,0.045138,0.087790,0.000000,0.090240,1.020865,0.084882,0.513023,...,0.317729,0.110418,0.000000,0.670983,0.736463,0.021563,0.108116,0.053033,0.021560,0.219832
netmarbleM.png,0.561076,0.222674,0.000000,0.000000,0.142209,0.000000,0.000000,0.984910,0.000000,0.060653,...,0.849142,0.945647,0.038702,0.366196,0.775896,0.014521,0.271726,0.011732,0.013724,0.429029


### 5.3 입력이미지 - 대표폰트 렌더링 이미지의 유사도 비교

In [112]:
rendering_1st_vec_sim = []
for i in range(len(rendering_1st_vector_df)):
  rendering_1st_vec_sim.append(cosine_similarity(test_img_vector_df.values, rendering_1st_vector_df.iloc[[i]].values)[0][0])

rendering_1st_vector_sim_df = rendering_1st_vector_df
rendering_1st_vector_sim_df['similarity'] = rendering_1st_vec_sim

### 5.4 대표폰트 중 입력이미지와 가장 유사한 폰트 도출

In [114]:
rendering_1st_vector_sim_df.reset_index(drop=False, inplace=True)
rendering_1st_vector_sim_df
most_sim_font_1st = rendering_1st_vector_sim_df.iloc[[rendering_1st_vector_df['similarity'].argmax()]].index[0]
most_sim_font_1st

3

-> 클러스터 3의 폰트가 입력이미지와 가장 유사함

## 6. 2차 렌더링

### 6.1 가장 유사한 대표폰트가 해당된 클러스터의 폰트로 렌더링

In [127]:
font_list_2nd_rendering = fonts_foreach_cluster.iloc[:, 3].dropna().to_numpy()

most_sim_cluster_font_list = []
for font in font_list_2nd_rendering:
  most_sim_cluster_font_list.append(font + '.ttf')

make_image("훈민정음2", most_sim_cluster_font_list)

### 6.2 렌더링한 텍스트 이미지의 특징벡터 추출

In [154]:
input_path = 'C:/Users/SM/Desktop/2022-2 졸업프로젝트/Similar font recommendation/훈민정음2/'
list_pics2 = []
filenames2 = []
for file2 in os.listdir(input_path):
    filename2 = os.fsdecode(file2)
    img2 = Image.open(os.path.join(input_path, filename2)).convert('RGB')
    list_pics2.append(img2)
    filenames2.append(filename2)
    
vectors2 = img2vec.get_vec(list_pics2)
pics2 = {}
for i2, vec2 in enumerate(vectors2):
    pics2[filenames2[i2]] = vec2
    
rendering_2st_vector_df = pd.DataFrame(pics2).transpose()
rendering_2st_vector_df

(33, 2048)


,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
20950870_경기천년바탕TTF_Bold.png,0.650108,0.132801,0.000000,0.027320,0.088512,0.000000,0.029281,0.524548,0.000000,0.048094,...,0.377864,0.894019,0.152225,0.166525,1.178108,0.013761,0.122153,0.059750,0.000222,0.370248
20950871_경기천년바탕TTF_Regular.png,0.595278,0.113723,0.000712,0.055307,0.136735,0.021906,0.058617,0.495630,0.005868,0.066591,...,0.609397,0.674796,0.137147,0.647842,0.821540,0.000000,0.098827,0.039885,0.009653,0.287346
20950938_Batang_TTF.png,0.400896,0.099663,0.023898,0.077975,0.034014,0.000000,0.013879,1.183862,0.015506,0.179762,...,0.463082,0.467527,0.000000,0.443978,0.902993,0.015316,0.051788,0.019863,0.000000,0.681924
20956073_SuncheonB_TTF.png,0.728799,0.258646,0.000000,0.016400,0.106561,0.015320,0.013570,0.688431,0.000000,0.111364,...,0.251531,1.370885,0.026965,0.116408,1.420308,0.014578,0.377735,0.077178,0.007801,0.128622
20958935_UTOIMAGE_유토이미지고딕_R_TTF.png,0.400110,0.203834,0.000000,0.008988,0.104647,0.006082,0.037024,0.598565,0.000000,0.050041,...,0.216519,0.701361,0.122988,0.288025,0.998957,0.000000,0.110673,0.103678,0.010676,0.143602
20958981_UTOIMAGE_유토이미지고딕_B_TTF.png,0.562646,0.295548,0.000000,0.010148,0.107855,0.000000,0.016006,0.518539,0.000000,0.043740,...,0.165702,0.946847,0.142334,0.223101,1.274979,0.011869,0.127993,0.087294,0.012197,0.197314
20958983_UTOIMAGE_유토이미지고딕_L_TTF.png,0.270689,0.134553,0.000000,0.029382,0.102065,0.000000,0.092429,0.376503,0.000000,0.191490,...,0.301210,0.498443,0.093131,0.597771,0.727167,0.003200,0.088908,0.115461,0.003588,0.074107
BinggraeⅡ.png,0.359707,0.159770,0.000000,0.062359,0.061286,0.000000,0.020364,0.769154,0.004326,0.233609,...,0.492846,0.475596,0.023715,0.448340,0.790239,0.046678,0.090830,0.008162,0.000000,0.537193
BMJUA_ttf.png,0.141892,0.242085,0.006217,0.044440,0.175719,0.000000,0.060020,0.662589,0.007342,0.225968,...,0.055700,1.276484,0.160865,0.347816,1.329986,0.012589,0.177363,0.232113,0.017489,0.309794
DXBrmRExtraBold-KSCpc-EUC-H.png,0.324385,0.271999,0.000000,0.044174,0.130532,0.002755,0.019988,0.961200,0.000000,0.201164,...,0.720190,0.771224,0.076198,0.139478,1.057356,0.014904,0.434095,0.217022,0.009038,0.420241


### 6.3 입력이미지 - 클러스터폰트 렌더링 이미지의 유사도 비교

In [155]:
rendering_2st_vec_sim = []
for i in range(len(rendering_2st_vector_df)):
  rendering_2st_vec_sim.append(cosine_similarity(test_img_vector_df.values, rendering_2st_vector_df.iloc[[i]].values)[0][0])

rendering_2st_vector_sim_df = rendering_2st_vector_df
rendering_2st_vector_sim_df['similarity'] = rendering_2st_vec_sim
rendering_2st_vector_sim_df

,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,similarity
20950870_경기천년바탕TTF_Bold.png,0.650108,0.132801,0.000000,0.027320,0.088512,0.000000,0.029281,0.524548,0.000000,0.048094,...,0.894019,0.152225,0.166525,1.178108,0.013761,0.122153,0.059750,0.000222,0.370248,0.733282
20950871_경기천년바탕TTF_Regular.png,0.595278,0.113723,0.000712,0.055307,0.136735,0.021906,0.058617,0.495630,0.005868,0.066591,...,0.674796,0.137147,0.647842,0.821540,0.000000,0.098827,0.039885,0.009653,0.287346,0.715909
20950938_Batang_TTF.png,0.400896,0.099663,0.023898,0.077975,0.034014,0.000000,0.013879,1.183862,0.015506,0.179762,...,0.467527,0.000000,0.443978,0.902993,0.015316,0.051788,0.019863,0.000000,0.681924,0.701866
20956073_SuncheonB_TTF.png,0.728799,0.258646,0.000000,0.016400,0.106561,0.015320,0.013570,0.688431,0.000000,0.111364,...,1.370885,0.026965,0.116408,1.420308,0.014578,0.377735,0.077178,0.007801,0.128622,0.787927
20958935_UTOIMAGE_유토이미지고딕_R_TTF.png,0.400110,0.203834,0.000000,0.008988,0.104647,0.006082,0.037024,0.598565,0.000000,0.050041,...,0.701361,0.122988,0.288025,0.998957,0.000000,0.110673,0.103678,0.010676,0.143602,0.709987
20958981_UTOIMAGE_유토이미지고딕_B_TTF.png,0.562646,0.295548,0.000000,0.010148,0.107855,0.000000,0.016006,0.518539,0.000000,0.043740,...,0.946847,0.142334,0.223101,1.274979,0.011869,0.127993,0.087294,0.012197,0.197314,0.747559
20958983_UTOIMAGE_유토이미지고딕_L_TTF.png,0.270689,0.134553,0.000000,0.029382,0.102065,0.000000,0.092429,0.376503,0.000000,0.191490,...,0.498443,0.093131,0.597771,0.727167,0.003200,0.088908,0.115461,0.003588,0.074107,0.653318
BinggraeⅡ.png,0.359707,0.159770,0.000000,0.062359,0.061286,0.000000,0.020364,0.769154,0.004326,0.233609,...,0.475596,0.023715,0.448340,0.790239,0.046678,0.090830,0.008162,0.000000,0.537193,0.722363
BMJUA_ttf.png,0.141892,0.242085,0.006217,0.044440,0.175719,0.000000,0.060020,0.662589,0.007342,0.225968,...,1.276484,0.160865,0.347816,1.329986,0.012589,0.177363,0.232113,0.017489,0.309794,0.682028
DXBrmRExtraBold-KSCpc-EUC-H.png,0.324385,0.271999,0.000000,0.044174,0.130532,0.002755,0.019988,0.961200,0.000000,0.201164,...,0.771224,0.076198,0.139478,1.057356,0.014904,0.434095,0.217022,0.009038,0.420241,0.797832


### 6.4 입력이미지와 가장 유사한 폰트 및 유사 폰트 추천 순위 도출

In [161]:
most_sim_font_2st = rendering_2st_vector_sim_df.iloc[[rendering_2st_vector_df['similarity'].argmax()]]
print('입력이미지와 가장 유사한 폰트 : ', most_sim_font_2st.index[0].replace('.png', ''))

### 추천 폰트 순위
rendering_2st_vector_sim_df.sort_values(by=['similarity'],ascending=False)

입력이미지와 가장 유사한 폰트 :  netmarbleB


,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,similarity
netmarbleB.png,0.488523,0.292578,0.000000,0.006936,0.062976,0.000000,0.000458,0.521571,0.154150,0.116917,...,1.315667,0.051227,0.097234,1.076288,0.014709,0.286448,0.083399,0.000000,0.451081,0.799054
DXBrmRExtraBold-KSCpc-EUC-H.png,0.324385,0.271999,0.000000,0.044174,0.130532,0.002755,0.019988,0.961200,0.000000,0.201164,...,0.771224,0.076198,0.139478,1.057356,0.014904,0.434095,0.217022,0.009038,0.420241,0.797832
20956073_SuncheonB_TTF.png,0.728799,0.258646,0.000000,0.016400,0.106561,0.015320,0.013570,0.688431,0.000000,0.111364,...,1.370885,0.026965,0.116408,1.420308,0.014578,0.377735,0.077178,0.007801,0.128622,0.787927
netmarbleM.png,0.350092,0.333751,0.000000,0.013519,0.070188,0.010301,0.000000,0.629764,0.080337,0.037748,...,0.932046,0.043230,0.316620,0.997818,0.032911,0.111313,0.036857,0.000000,0.224602,0.783604
타이포도담체.png,0.680843,0.237579,0.000000,0.025473,0.090078,0.000000,0.043004,0.883795,0.026450,0.000973,...,1.045303,0.124127,0.149218,1.175505,0.016652,0.158981,0.010348,0.000000,0.357187,0.776154
THE명품굴림B.png,0.537331,0.366766,0.000000,0.012684,0.185957,0.000000,0.021697,0.577982,0.000000,0.034541,...,1.353674,0.129321,0.187367,1.356461,0.000000,0.163467,0.126572,0.045709,0.177977,0.760929
DXMgoRB-KSCpc-EUC-H.png,0.868777,0.166382,0.003870,0.000000,0.048768,0.000000,0.068777,0.533590,0.031446,0.064102,...,1.504709,0.188028,0.672077,1.412203,0.001159,0.090617,0.131155,0.027124,0.239151,0.758867
20958981_UTOIMAGE_유토이미지고딕_B_TTF.png,0.562646,0.295548,0.000000,0.010148,0.107855,0.000000,0.016006,0.518539,0.000000,0.043740,...,0.946847,0.142334,0.223101,1.274979,0.011869,0.127993,0.087294,0.012197,0.197314,0.747559
휴먼옛체.png,0.510288,0.236995,0.000000,0.028482,0.072449,0.004044,0.020342,0.802682,0.013487,0.065370,...,1.270799,0.146290,0.287975,1.144304,0.053366,0.178710,0.210372,0.022610,0.136887,0.743855
OSeongandHanEum-Bold.png,0.180688,0.256078,0.000000,0.046791,0.164895,0.007102,0.012410,0.693239,0.000000,0.324455,...,0.984195,0.138714,0.032483,1.150054,0.001678,0.180489,0.074039,0.002402,0.522099,0.743662
